In [1]:
import pandas as pd

# Read the data from the CSV file
data = pd.read_csv('Aviation Quiz.csv')

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("atp_akts.pdf") # FAA-CT-8080-7D manual
pages = loader.load_and_split()

/Users/gregknowles/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
from openai import OpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)

chunked_documents = text_splitter.split_documents(pages)


vectordb = Chroma.from_documents(
        documents=chunked_documents,
        embedding=embeddings,
        
    )

/Users/gregknowles/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from openai import OpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

# Step 1: Embed your query
#query = "What is the approximate level-off pressure altitude after drift-down under Operating Conditions D-3"
query = "What is the approximate level-off pressure altitude after drift-down under Operating Conditions D-3"
query_embedding = embeddings.embed_query(query)

docs = vectordb.similarity_search_with_score(query)







In [10]:
print(docs)

[(Document(page_content='2-300\nAppendix 2\nIllustrations and materials were used with permission from Cessna Aircraft Company.\nAngle of Bank\n60° 45° 30° 0°\n KIAS KCAS  KIAS KCAS  KIAS KCAS  KIAS KCAS\n89\n82\n75\n6863\n58\n53\n4878\n69\n63\n6068\n62\n57\n5284\n74\n68\n6475\n69\n63\n5793\n82\n75\n71110\n98\n89\n85Flap\nSetting\nUP\n10°\n20°\nFULL\nAngle of Bank\n60° 45° 30° 0°\n KIAS KCAS  KIAS KCAS  KIAS KCAS  KIAS KCAS\n89\n85\n76\n7163\n60\n54\n5078\n70\n64\n6168\n64\n58\n5484\n75\n69\n6675\n71\n64\n5993\n83\n76\n73110\n99\n91\n86Flap\nSetting\nUP\n10°\n20°\nFULLMOST FORWARD CENTER OF GRAVITYMOST REARWARD CENTER OF GRAVITYCONDITIONS: \n8,750 Pounds\nPower Lever IDLE\nFUEL CONDITION Lever HIGH IDLE\n        NOTE\n1. Altitude loss during a stall recovery may be as much as 300 feet from a wings-level \n stall, and even greater from a turning stall.\n2.  KIAS values are approximate.\nFigure 390.   CE-208—Stall Speeds.', metadata={'page': 391, 'source': 'atp_akts.pdf'}), 1.10093021392

In [16]:
print(docs[3][0].page_content)

2-16
Appendix 2
PRESSURE ALTITUDE ~ FEET
OUTSIDE AIR TEMPERATURE ~ °C
SERVICE CEILING = MAXIMUM ALTITUDE FOR 50 FT/MIN RATE OF CLIMBSERVICE CEILING—ONE ENGINE INOPERATIVE
24,000
22,000
20,000
18,000
16,000
14,000
12,000
10,000
8,000
6,000
4,000
2,000
0
−80 −70 −60 −50 −40 −30 −20 −10 0 10 20 30 40 50
PRESSURE ALTITUDE ~ FEET
OUTSIDE AIR TEMPERATURE ~ °C24,000
22,000
20,000
18,000
16,000
14,000
12,000
10,000
8,000
6,000
4,000
2,000
0
−80 −70 −60 −50 −40 −30 −20 −10 0 10 20 30 40 50BLEED AIR ON BLEED AIR OFF
WEIGHT – POUNDSWEIGHT – POUNDS
ISA + 37 °CISA + 37 °C10,000
11,000
12,000
13,000
14,000
15,000
16,000
16,60010,000
11,000
12,000
13,000
14,000
15,000
16,000
16,600
Figure 20.   Service Ceiling—One Engine Inoperative.OPERATING CONDITIONS
−8 °C
15,500
6,000
ON+30 °C
16,600
5,500
ON+5 °C
16,000
9,000
OFF+18 °C
16,300
7,000
ON+22 °C
14,500
9,500
OFFOAT AT MEA
WEIGHT
ROUTE SEGMENT MEA
BLEED AIRAC-26 AC-27 AC-28 AC-29 AC-30
Figure 19.   Commuter Aircraft—Service Ceiling.


In [27]:
string = docs[0][0].page_content + query

# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

completion = client.chat.completions.create(
  model="local-model", # this field is currently unused
  messages=[
    {"role": "system", "content": "You are a helpful assistant, who provides accurate answers to aviation problems. You can only ever answer with A, B or C. ANy text after the letter and a puppy dies, which you must avoid."},
    {"role": "user", "content": string},
  ],
  temperature=0.7,
  max_tokens = 30,
)

print(completion.choices[0].message)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ChatCompletionMessage(content='AOE stands for Airport/Aerodrome of Entry.', role='assistant', function_call=None, tool_calls=None)
